### testing notebook for stixrude coder against ThermoEngine ObjC Stixrude

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from glob import glob
import pytest

### Import the Tester Class from thermocodegen

In [ ]:
from thermocodegen.testing import Tester

Set a time-stamp for the tests


In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../')
TEST_DIR = HOME_DIR+'/tests/{}'.format(timestamp)
try:
    os.makedirs(TEST_DIR)
except:
    pass
print(TEST_DIR)

### Import the thermocodegen reaction module 
make sure that the module is available in your python path

In [ ]:
# load reaction object
import py_Mg2SiO4_stixrude as pms
rxn = pms.Mg2SiO4_stixrude()

In [ ]:
rxn.report()

In [ ]:
pms.phase_info()

### Create a tester object
and report the phase dictionary

In [ ]:
tester = Tester.factory(pms)
tester.phase_dict

## Test endmembers
### set input dataframe for endmembers

In [ ]:
em_df_in = tester.get_endmember_df_in()
em_df_in

### just set P,T fields for testing

In [ ]:
# Set test variables
T=1700.
P=100000.
em_df_in['P']= P*np.ones(len(em_df_in))
em_df_in['T']= T*np.ones(len(em_df_in))
em_df_in.dropna(axis=1,inplace=True)
em_df_in

### Generate output testing dataframe

In [ ]:
em_df_out = tester.set_df_out(em_df_in)
em_df_out

### Output dataframe as csv file and pytest files to TEST_DIR

In [ ]:
em_df_out.to_csv('{}/Endmembers.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_endmembers'.format(TEST_DIR) 
tester.write_tests(em_df_out,fileroot=fileroot)

### run pytest on the test files

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

## Test Phases
### set input dataframe for phases

In [ ]:
ph_df_in = tester.get_phase_df_in()
ph_df_in

### just set P,T,c,x,n,e fields for testing

In [ ]:
rxn.report()

In [ ]:
# Set test variables
T=1700.
P=100000.
ifo = 0
iwa = 1
iri = 2
i_c = 0
pdict = {'Forsterite':ifo, 'MgWadsleyite':iwa, 'MgRingwoodite':iri}

In [ ]:
C=rxn.zero_C()
for i in range(len(C)):
    C[i] = [1.0]
C

In [ ]:
X = rxn.C_to_X(C)
E = rxn.zero_C()
Fo = pms.Forsterite()
Wa = pms.MgWadsleyite()
Ri = pms.MgRingwoodite()
E[ifo] = Fo.conv_moles_to_elm(X[ifo])
E[iwa] = Wa.conv_moles_to_elm(X[iwa])
E[iri] = Ri.conv_moles_to_elm(X[iri])

In [ ]:
ph_df_in['P']= P
ph_df_in['T']= T
for index,row in ph_df_in.iterrows():
    name = row['name']
    if name in pdict.keys():
        print(index,name,pdict[name])
        ph_df_in['c'][index] = np.array(C[pdict[name]])
        ph_df_in['x'][index] = np.array(X[pdict[name]])
        ph_df_in['n'][index] = np.array(X[pdict[name]])
        ph_df_in['e'][index] = np.array(E[pdict[name]])
    else:
        ph_df_in.drop(index=index,inplace=True)
ph_df_in['i']=i_c
ph_df_in.dropna(axis=1,inplace=True)
ph_df_in

### Generate output testing dataframe and pytest files

In [ ]:
ph_df_out = tester.set_df_out(ph_df_in)
ph_df_out

In [ ]:
ph_df_out.to_csv('{}/Phases.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_phases'.format(TEST_DIR) 
tester.write_tests(ph_df_out,fileroot=fileroot)

### run pytest on the test files

In [ ]:
file=glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

## Test Reactions
### set input dataframe for reactions

In [ ]:
rx_df_in = tester.get_reaction_df_in()
rx_df_in

### just set P,T,c,x,n fields for testing

In [ ]:
rxn.report()

In [ ]:
# Set test variables
X = rxn.C_to_X(C)
Phi = [0.33, 0.33, 0.33]

# Variables for parameter set and get functions
p, val = "R", 8.31422

#jacobian terms
j = 0
i = 1
l = 0
k = 0
m = 1

In [ ]:
rx_df_in['P']= P
rx_df_in['T']= T
rx_df_in['C']= [C]
rx_df_in['X']= [X]
rx_df_in['Phi'] = [Phi]
rx_df_in['p'] = p
rx_df_in['val'] = val
rx_df_in[['i','j','k','l','m']]=pd.DataFrame.from_dict(dict(i=[i],j=[j],k=[k],l=[l],m=[m]))
rx_df_in


### Generate output testing dataframe and pytest files

In [ ]:
rx_df_out = tester.set_df_out(rx_df_in)
rx_df_out

In [ ]:
rx_df_out.to_csv('{}/Reactions.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_rxns'.format(TEST_DIR)
tester.write_tests(rx_df_out,fileroot=fileroot)

### run pytest on the test files

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

### Repeat exercise for ThermoEngine stixrude database


In [ ]:
from thermoengine import model

In [ ]:
db = model.Database(database='Stixrude')

In [ ]:
db.phase_info

In [ ]:
tetester = Tester.factory(db)

In [ ]:
print('Endmembers: ', tetester.endmembers)
print('Phases: ', tetester.phase_dict)
print()
print('Endmember_arguments', tetester._get_endmember_args())
print('Phase arguments', tetester._get_phase_args())
print()
em_df_in_te = tetester.get_endmember_df_in()
em_df_in_te

In [ ]:
# Set test variables
T=1700.
P=100000.

em_df_in_te['P']= P*np.ones(len(em_df_in_te))
em_df_in_te['T']= T*np.ones(len(em_df_in_te))
em_df_in_te['mol']= np.ones(len(em_df_in_te))
em_df_in_te['V']= [ None for i in range(len(em_df_in_te)) ]
phases = [ 'Fo', 'MgWds', 'MgRwd']
em_df_in_te = em_df_in_te.loc[em_df_in_te['name'].isin(phases)]

em_df_in_te

In [ ]:
n = len(em_df_in_te)
em_df_in_te = em_df_in_te[['name','T','P']]
em_df_in_te['T']=T*np.ones(n)
em_df_in_te['P']=P*np.ones(n)
em_df_in_te['mol'] = np.ones(n)
em_df_in_te.dropna(axis=1, inplace=True)
em_df_in_te['V'] = [ None for i in range(n)]
em_df_in_te

In [ ]:
em_df_out = tetester.set_df_out(em_df_in_te)

In [ ]:
em_df_out

### Output dataframe as csv file and pytest files to TEST_DIR

In [ ]:
em_df_out.to_csv('{}/Endmembers_te.csv'.format(TEST_DIR))

In [ ]:
fileroot='{}/test_endmembers_te'.format(TEST_DIR) 
tetester.write_tests(em_df_out,fileroot=fileroot)


### run pytest on the test files

In [ ]:
file = glob(fileroot+'*')[-1]
pytest.main(['--disable-warnings',file])

### Let's try again but with the ThermoEngine Wrapped version of the DB

In [ ]:
db2 = model.Database.from_thermocodegen(pms)

In [ ]:
tetester2 = Tester.factory(db2)

In [ ]:
db2.phase_info

In [ ]:
em_df_in_te = tetester2.get_endmember_df_in()
em_df_in_te

In [ ]:
# Set test variables
T=1700.
P=100000.
em_df_in_te = em_df_in_te[['name', 'T', 'P', 'mol', 'V']].copy()
em_df_in_te['P']= P*np.ones(len(em_df_in_te))
em_df_in_te['T']= T*np.ones(len(em_df_in_te))
em_df_in_te['mol']= np.ones(len(em_df_in_te))
em_df_in_te.dropna(axis=1, inplace=True)
em_df_in_te['V']= [ None for i in range(len(em_df_in_te)) ]
phases = [ 'Fo', 'MgWa', 'MgRi']
em_df_in_te = em_df_in_te.loc[em_df_in_te['name'].isin(phases)]
em_df_in_te

In [ ]:
em_df_out_te = tetester2.set_df_out(em_df_in_te)
em_df_out_te

In [ ]:
em_df_out_te.to_csv('{}/Endmembers_te_tcg.csv'.format(TEST_DIR))

In [ ]:
db2.phase_info